# Curva Elíptica

In [343]:
NIST = dict()
NIST['P-192'] =  {
    'p': 6277101735386680763835789423207666416083908700390324961279,
    'n': 6277101735386680763835789423176059013767194773182842284081,
    'seed' : '3045ae6fc8422f64ed579528d38120eae12196d5',
    'c': '3099d2bbbfcb2538542dcd5fb078b6ef5f3d6fe2c745de65',
    'b': '64210519e59c80e70fa7e9ab72243049feb8deecc146b9b1',
    'Gx' : '188da80eb03090f67cbf20eb43a18800f4ff0afd82ff1012',
    'Gy' : '07192b95ffc8da78631011ed6b24cdd573f977a11e794811'
}

USED_NONCES = []

# Classe que implementa o ECDSA

In [344]:
class ECDSA:
    def __init__(self):
        self = self.setup()
        
    def setup(self):
        c = NIST['P-192']
        p  = c['p'] 
        n  = c['n']
        b  = ZZ(c['b'],16)
        Gx = ZZ(c['Gx'],16)
        Gy = ZZ(c['Gy'],16)
        self.E = EllipticCurve(GF(p),[-3,b])
        self.q = E.order()
        self.N = GF(q) # Zq
        self.G = E((Gx,Gy))
        return self
    
    def gen_key_pair(self):
        d = ZZ(self.N.random_element()) # private key
        Q = d*self.G                    # public key
        return d, Q
    
    def sign(self, m, d):
        r = 0; s = 0
        while r == s == 0:
            k = ZZ.random_element(1, self.q)
            if not k in USED_NONCES:
                r = N((k*self.G).xy()[0])
                s = N((m+d*r)/N(k))
            USED_NONCES.append(k)
        return r, s
    
    def verify(self, m, r, s, Q):
        w = 1/s
        ri = ZZ(r*w)
        mi = ZZ(m*w)
        if r == N((mi*self.G + ri*Q).xy()[0]):
            print 'Valid signature'
        else:
            print 'Invalid signature'

# Exemplo de utilização

In [349]:
ecdsa = ECDSA()

# Generate key pair
private_key, public_key = ecdsa.gen_key_pair()

# Message
message = ZZ(N.random_element()) # HASH?

# Sign
ephemeral_key, signature = ecdsa.sign(message, private_key)

# Verify
ecdsa.verify(message, ephemeral_key, signature, public_key)

Valid signature
